In [1]:
import sys
import Spytrometer
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# Initialize pycomet object
spy = Spytrometer.Spytrometer(max_peak=2000)
bin_width = 0.02
bin_offset = 0

spy.tolarence_window = 10
spy.tolerance_type = 'PPM'
spy.max_theo_pept_peak_charge = 3
spy.remove_precursor_peak = True
spy.remove_precursor_tolerance = 1.5
spy.missed_cleavages = 0
spy.min_pept_len = 7
spy.unique_peptides = 1

In [3]:
# Load data
start_time = datetime.now()
spy.load_dia_data('c:\\Users\\poder\\crux-demo\\Slider_DIA\\e01306.mzML', precursor_mass = 410.4365)
spy.load_fasta('c:\\Users\\poder\\crux-demo\\cerevisiae_orf_trans_all.fasta')

Loading spectrum data...
Done. Time (h:m:s):	0:00:48.430796


In [4]:
for spectrum_id, spectrum in enumerate(spy.spectrum_collection):
    spy.discretize_spectrum(spectrum_id)
    spy.normalize_regions(spectrum_id)
for i in range(len(spy.protein_collection)):
    try:
        spy.generate_peptides(i)
    except:
        continue
spy.sort_peptides()    
spy.sort_spectra()
spy.set_candidate_peptides()      
print("Protein digestion done. Time (h:m:s):\t"+str(datetime.now() - start_time))
print("Number of proteins:\t%d"%(len(spy.protein_collection)))
print("Number of spectra:\t%d"%(len(spy.spectrum_collection)))
print("Number of peptides:\t%d"%(len(spy.peptide_collection)))

Protein digestion done. Time (h:m:s):	0:01:05.946842
Number of proteins:	6717
Number of spectra:	1790
Number of peptides:	134349


In [5]:
# Load train_data obtained by percolator
data_percolator = pd.read_csv('c:\\Users\\poder\\crux-demo\\Slider_DIA\\train_data_percolator.csv')
print(len(data_percolator))

180


In [6]:
# Set the index of pair (scan_id, sequence) from train_data_percolator.csv which you want to visualize
percolator_idx = -1

In [7]:
percolator_scan_id = data_percolator.iloc[percolator_idx].scan
percolator_sequence = data_percolator.iloc[percolator_idx].sequence
print(percolator_scan_id, percolator_sequence)

60477 DAGQIVGLNVLR


In [8]:
# Search for the index of percolator_sequence in spy.peptide_collection
for i in range(len(spy.peptide_collection)):
    if spy.peptide_collection[i].peptide_seq == percolator_sequence:
        peptide_idx = i
print(peptide_idx)

58702


In [9]:
# Search for the index of percolator_scan_id in spy.spectrum_collection
spectrum_idx = spy.get_spectrum_by_scan(percolator_scan_id - 1)
print(spectrum_idx)

1475


In [10]:
# Get peaks for PeptideObj
spy.calculate_peptide_fragmentation(peptide_idx)
print(spy.peptide_collection[peptide_idx].peaks)

[{'b': [116, 187, 244, 372, 485, 584, 641, 754, 868, 967, 1080], 'y': [175, 288, 387, 501, 614, 671, 770, 883, 1011, 1068, 1139]}, {'b': [59, 94, 123, 187, 243, 293, 321, 378, 435, 484, 541], 'y': [88, 145, 194, 251, 308, 336, 386, 442, 506, 535, 570]}, {'b': [39, 63, 82, 125, 162, 195, 214, 252, 290, 323, 361], 'y': [59, 97, 130, 168, 205, 224, 257, 295, 338, 357, 380]}]


In [12]:
spy.plot_spectrum(spectrum_idx, filename="Plot_{}".format(spectrum_idx), font_size=18, show_annotation=True, peptide_seq=spy.peptide_collection[peptide_idx])

Seq:DAGQIVGLNVLR  Charge:1   Mass: 1253.7091547
